### Limpeza da base de dados de notas fiscais


### Descrição das Colunas - Base de Notas Fiscais

| Nome da Coluna           | Descrição                                                                                  | Exemplo                                                                 |
|--------------------------|-------------------------------------------------------------------------------------------|-------------------------------------------------------------------------|
| **Numero nota fiscal**   | Identificador único da nota fiscal (documento fiscal).                                    | `566082`                                                               |
| **Data da venda**        | Data de emissão da nota fiscal (formato DD/MM/AAAA).                                      | `17/06/2025`                                                           |
| **Código produto**       | Código único do produto vendido (deve corresponder ao `Código` na base de estoque).       | `33932`, `241`                                                         |
| **Descrição do produto** | Nome/descrição do produto conforme registrado na nota fiscal.                             | `"ENV 229X324 OU C/100 FILIPERSON"`, `"PASTA AZ OF LE TIGRADA FRAMA"`  |
| **Quantidade do produto**| Quantidade vendida do produto (unidades, caixas, kg, etc.).                              | `2.0`, `40.0`                                                          |
| **Valor unitário**       | Preço unitário do produto no momento da venda (sem descontos/acréscimos).                | `33.000000`, `9.450000`                                                |
| **Preço venda**          | Preço mínimo de venda sugerido.                  | `27.476514`, `8.863195`                                                |
| **Preço de custo**       | Custo unitário do produto no momento da compra (o custo da empresa para adquirir).                  | `24.409041`, `7.605561`                                                |
| **Valor da nota**        | Valor total da nota fiscal (somatório de todos os itens + impostos/fretes, se aplicável).| `3895.94`                                                              |


In [ ]:
import pandas as pd

In [ ]:
df_vendas = pd.read_excel('bases/relatorio_notas.xlsx')

### Propriedades da base

In [ ]:
print(f"Formato dos dados {df_vendas.shape}")

In [ ]:
len(df_vendas["Numero nota fiscal"].unique())

In [ ]:
df_vendas.columns.tolist()

In [ ]:
df_vendas.head()

In [ ]:
df_vendas.dtypes

In [ ]:
df_vendas.describe()

In [ ]:
print("Quantidade valores Nulos")
df_vendas.isnull().sum()

In [ ]:
print("Quantidade de valores zerados")
colunas_numericas = df_vendas.select_dtypes(include='number')
for coluna in colunas_numericas:
    at_zeros = (colunas_numericas[coluna] == 0).sum()
    print(f"\n{coluna} contém {at_zeros} zeros")



In [ ]:
print("Quantidade de valores negativos")
for rotulo in colunas_numericas:
    qt_negativos = (colunas_numericas[rotulo] < 0).sum()
    print(f"\n{rotulo} contém {qt_negativos} negativos")

### Análise de Consistência e Decisões - Base de Vendas

#### **Regras de Negócio Validadas**
| Regra | Descrição | Impacto no Modelo |
|-------|-----------|-------------------|
| **Cálculo do Valor por Item** | `Valor unitário` × `Quantidade produto` = Valor total do produto na nota. | Garante a correta agregação dos valores por item. |
| **Cálculo do Valor da Nota** | Somatório do valor total de todos os itens = `Valor da nota`. | Notas com divergência serão sinalizadas para revisão. |
| **Exclusão de Campo** | `Preço venda` será removido do modelo por ser apenas uma sugestão comercial. | Reduz ruídos e simplifica a análise de preços reais. |

---

#### **Problemas Identificados e Impactos**

1. **Preço de Custo Nulo**  
   - **Ocorrências**: 2.224 registros sem `Preço de custo`.  
   - **Análise**:  
     - Produtos, categorias e marcas afetadas serão priorizadas por volume de vendas.  
     - Impacta o cálculo de margem de lucro para **itens em múltiplas notas fiscais**.  

2. **Notas com Valores Zerados**  
   | Campo Zerado | Ocorrências | Impacto |
   |-------------|------------|---------|
   | `Quantidade produto` | 1.749 | Itens inválidos (exclusão necessária). |
   | `Valor unitário` | 2.017 | Distorção no valor total da nota. |  
   - **Decisão**: Registros com esses campos zerados serão **excluídos** do modelo.  

3. **Valores Negativos**  
   - **Ocorrências**: 459 registros com `Quantidade produto` negativa.  
   - **Ação**: Serão excluidos da base, mas **exportados** para serem analisados separadamente.  

---

#### **Resumo de Ações para o Modelo**
| Etapa | Decisão | Justificativa |
|-------|---------|---------------|
| **Padronização** | Validar `Valor da nota` vs. somatório dos itens. | Garantir integridade financeira. |
| **Limpeza** | Excluir registros com `Quantidade produto` ≤ 0 ou `Valor unitário` = 0. | 0.5% total de notas impactadas Evitar distorções nas recomendações. |
| **Substituição** | Preencher `Preço de custo` nulo com valores da base de estoque. | Permitir cálculo de margem. |
| **Exclusão** | Remover `Preço venda` (campo não confiável). | Foco em dados reais de transação. |

In [ ]:
# Filtrar apenas as linhas com quantidade negativa
df_negativos = df_vendas[df_vendas['Quantidade do produto'] < 0]

# Contar produtos únicos e notas fiscais impactadas
total_produtos_negativos = df_negativos['Descrição do produto'].nunique()
total_notas_impactadas = df_negativos['Numero nota fiscal'].nunique()

# Agrupar por produto para ver a distribuição
analise_produtos = df_negativos.groupby('Descrição do produto').agg(
    Total_Quantidade_Negativa=('Quantidade do produto', 'sum'),
    Ocorrencias=('Quantidade do produto', 'count')
).sort_values('Total_Quantidade_Negativa')

print(f"🔍 {len(df_negativos)} registros negativos afetam {total_notas_impactadas} notas fiscais e {total_produtos_negativos} produtos distintos.\n")
print("📊 Produtos com quantidades negativas:")
display(analise_produtos)

In [ ]:
# gardando base com valores negativos para uma análise posterior
analise_produtos.to_excel("produtos_com_quantidade_negativa.xlsx")

In [ ]:
# Contagem para 'Quantidade do produto' zerada
qtde_zeros = (df_vendas['Quantidade do produto'] == 0).sum()
produtos_qtde_zero = df_vendas[df_vendas['Quantidade do produto'] == 0]['Quantidade do produto'].nunique()
notas_qtde_zero = df_vendas[df_vendas['Quantidade do produto'] == 0]['Numero nota fiscal'].nunique()

# Contagem para 'Valor unitário' zerado
valor_zeros = (df_vendas['Valor unitário'] == 0).sum()
produtos_valor_zero = df_vendas[df_vendas['Valor unitário'] == 0]['Quantidade do produto'].nunique()
notas_valor_zero = df_vendas[df_vendas['Valor unitário'] == 0]['Numero nota fiscal'].nunique()

# Resultado
print("📊 Análise de Valores Zerados:")
print(f"\n1. Quantidade do produto:")
print(f"- Registros zerados: {qtde_zeros}")
print(f"- Produtos impactados: {produtos_qtde_zero}")
print(f"- Notas fiscais impactadas: {notas_qtde_zero}")

print(f"\n2. Valor unitário:")
print(f"- Registros zerados: {valor_zeros}")
print(f"- Produtos impactados: {produtos_valor_zero}")
print(f"- Notas fiscais impactadas: {notas_valor_zero}")

In [ ]:
# 1. Identificar notas fiscais problemáticas
notas_com_problemas = set()

# Notas com quantidades negativas
notas_com_problemas.update(
    df_vendas[df_vendas['Quantidade do produto'] < 0]['Numero nota fiscal'].unique()
)

# Notas com quantidades zeradas
notas_com_problemas.update(
    df_vendas[df_vendas['Quantidade do produto'] == 0]['Numero nota fiscal'].unique()
)

# Notas com valores unitários zerados
notas_com_problemas.update(
    df_vendas[df_vendas['Valor unitário'] == 0]['Numero nota fiscal'].unique()
)

# 2. Filtrar para manter APENAS notas SEM problemas
df_venda_limpo = df_vendas[~df_vendas['Numero nota fiscal'].isin(notas_com_problemas)].copy()

# 3. Análise de impacto
total_notas_originais = df_vendas['Numero nota fiscal'].nunique()
total_notas_limpas = df_venda_limpo['Numero nota fiscal'].nunique()
notas_removidas = total_notas_originais - total_notas_limpas

print(f"📊 Notas fiscais removidas: {notas_removidas}/{total_notas_originais} ({notas_removidas/total_notas_originais:.1%})")
print(f"📦 Registros originais: {len(df_vendas)}")
print(f"🧹 Registros após limpeza: {len(df_venda_limpo)}")
print(f"🚮 Registros removidos: {len(df_vendas) - len(df_venda_limpo)}")

In [ ]:
df_venda_limpo.nunique()

In [ ]:
# 1. Verificar códigos com múltiplas descrições (problema grave)
codigos_problematicos = df_vendas.groupby('Código produto')['Descrição do produto'].nunique()
codigos_com_erro = codigos_problematicos[codigos_problematicos > 1]

print(f"🚨 {len(codigos_com_erro)} códigos com múltiplas descrições:")
display(codigos_com_erro.head())

# 2. Verificar códigos sem descrição (ausência de cadastro)
codigos_sem_descricao = df_vendas[df_vendas['Descrição do produto'].isna()]['Código produto'].nunique()
print(f"\n🔍 {codigos_sem_descricao} códigos sem descrição cadastrada.")

# 3. Análise de correspondência
print("\n📊 Relação Código-Descrição:")
print(f"- Códigos únicos: {df_vendas['Código produto'].nunique()}")
print(f"- Descrições únicas: {df_vendas['Descrição do produto'].nunique()}")
print(f"- Diferença: {df_vendas['Código produto'].nunique() - df_vendas['Descrição do produto'].nunique()}")

In [ ]:
# Agrupa descrições e conta quantos códigos únicos cada uma tem
descricoes_problematicas = df_vendas.groupby('Descrição do produto')['Código produto'].nunique()
descricoes_problematicas = descricoes_problematicas[descricoes_problematicas > 1].index.tolist()

print(f"🔍 {len(descricoes_problematicas)} descrições associadas a múltiplos códigos.")

In [ ]:
# Filtra o DataFrame para manter apenas registros com descrições problemáticas
df_problema = df_vendas[df_vendas['Descrição do produto'].isin(descricoes_problematicas)]

In [ ]:
# Conta quantas notas fiscais únicas contêm esses registros
notas_impactadas = df_problema['Numero nota fiscal'].nunique()
total_notas = df_vendas['Numero nota fiscal'].nunique()

print(f"📊 Notas fiscais impactadas: {notas_impactadas}/{total_notas} ({notas_impactadas/total_notas:.1%})")

In [ ]:
df_problema.to_excel("notas_com_descricoes_ambiguas.xlsx", index=False)

In [ ]:
# 1. Encontre descrições associadas a múltiplos códigos
descricoes_ambiguas = df_vendas.groupby('Descrição do produto')['Código produto'].nunique()
descricoes_ambiguas = descricoes_ambiguas[descricoes_ambiguas > 1].index.tolist()

# 2. Filtre as notas fiscais que contêm essas descrições
notas_com_problemas = df_vendas[df_vendas['Descrição do produto'].isin(descricoes_ambiguas)]['Numero nota fiscal'].unique()

In [ ]:
# 3. Crie um novo DataFrame EXCLUINDO as notas problemáticas
df_vendas_limpo = df_vendas[~df_vendas['Numero nota fiscal'].isin(notas_com_problemas)].copy()

In [ ]:
# 4. Calcule o impacto
total_notas_originais = df_vendas['Numero nota fiscal'].nunique()
total_notas_limpas = df_vendas_limpo['Numero nota fiscal'].nunique()
notas_removidas = total_notas_originais - total_notas_limpas

print(f"🔍 {len(descricoes_ambiguas)} descrições ambíguas encontradas.")
print(f"📊 Notas fiscais removidas: {notas_removidas}/{total_notas_originais} ({notas_removidas/total_notas_originais:.2%})")
print(f"📦 Registros originais: {len(df_vendas)}")
print(f"🧹 Registros após limpeza: {len(df_vendas_limpo)}")

In [ ]:
# Tratar o valor da nota, somatório de quantidade de produto * valor unitário
# calcular o valor total de cada produto por nota
df_vendas_limpo["Valor total produto"] = (df_venda_limpo["Quantidade do produto"] * df_venda_limpo["Valor unitário"]).round(2)

In [ ]:
# Passo 1: Calcular o valor total por nota fiscal
valor_por_nota = df_vendas_limpo.groupby('Numero nota fiscal')['Valor total produto'].sum().round(2)

# Passo 2: Mapear esse valor de volta para a coluna 'Valor da nota'
df_vendas_limpo['Valor da nota'] = df_vendas_limpo['Numero nota fiscal'].map(valor_por_nota)

# Verificação (opcional)
print("✅ Valores atualizados com sucesso!")
print(f"Total de notas fiscais processadas: {len(valor_por_nota)}")

In [ ]:
df_vendas_limpo = df_vendas_limpo[["Numero nota fiscal", "Data da venda", "Código produto", "Descrição do produto", "Quantidade do produto", "Valor unitário", "Preço de custo", "Valor total produto", "Valor da nota"]]

In [ ]:
# Arredonda as colunas específicas para 2 casas decimais
df_vendas_limpo['Valor unitário'] = df_vendas_limpo['Valor unitário'].round(2)
df_vendas_limpo['Preço de custo'] = df_vendas_limpo['Preço de custo'].round(2)

In [ ]:
df_vendas_limpo.to_excel("bases/bases_limpas/base_notas_limpa.xlsx", index=False)